In [19]:
## coding: utf-8
# データベースを作る
# 計測用信号を読み込む
# 測位する



import wave
import scipy as sp
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
from scipy.fftpack import fft
import scipy.signal as sg
from scipy import signal
import pandas as pd
import scipy.interpolate as itpl
import resampy

def create_db():
    sound_db = np.zeros(144*7*91*81).reshape(91,81,144*7)
    sound_db[:,:,:] = np.nan
    speaker_deg = 0
    for j in range(0,4):
        mic_deg = -40
        for i in range(0,9):
            #整形済み信号読み込み
            signal = np.loadtxt('s'+str(speaker_deg)+'m'+str(mic_deg)+'.txt')
            signal_fft = np.abs(fft(signal))
            #chirps_up =  resampy.resample(signal_fft, 144, 144*2, filter='kaiser_fast')
            sound_db[speaker_deg,mic_deg+40,:] = signal_fft
            mic_deg += 10
        speaker_deg +=30
    #ここから秋間補間，マイク方位で補間した後スピーカ角度で補間
    Akima = np.zeros(144*7*91*81).reshape(91,81,144*7)
    Akima[:,:,:] = np.nan
    #マイク方位で補間
    count = 0
    for i in range(0,4):
        sound_db_d = pd.DataFrame(sound_db[count,:,:])
        sound_db_d.astype('float64')
        Akima[count,:,:] = sound_db_d.interpolate('akima')
        count += 30
    #スピーカ角度で補間
    for j in range(0,81):
        sound_db_d = pd.DataFrame(Akima[:,j,:])
        sound_db_d.astype('float64')
        Akima[:,j,:] = sound_db_d.interpolate('akima')
    #Akima = pd_Zrf_d.interpolate('akima')
    print('Akima done!!')
    return Akima
    
# 残差平方和
def rss(y, t):
    f = np.sum((y-t).T*(y-t))
    return f
    
    
def estimate(db,file):
    #データベースを読み込んで、残差平方和を計測
    rss_db = np.zeros(91*81).reshape(91,81)
    #print(rss_db)
    for j in range(0,91):
        for i in range(0,81):
            rss_ans = rss(db[j,i],file[:])
            rss_db[j,i] = rss_ans
    estimate_mic_speaker = np.unravel_index(np.argmin(rss_db), rss_db.shape)
    estimate_mic_speaker = list(estimate_mic_speaker)
    estimate_mic_speaker[1] -= 40
    print('(推定スピーカー角度,推定マイク角度)'+str(estimate_mic_speaker))
    
if __name__ == '__main__':
    db1 = create_db()
    #整形済み測位信号読み込み
    signal = np.loadtxt('s45m-25.txt')
    chirps_up1 = np.abs(fft(signal))
    #chirps_up1 =  resampy.resample(np.abs(fft(signal)), 144, 144*2, filter='kaiser_fast')
    # 最小二乗法によって測位
    estimate(db1,chirps_up1)

Akima done!!
(推定スピーカー角度,推定マイク角度)[43, -34]


In [ ]:
0,-25
(推定スピーカー角度,推定マイク角度)[0, -28]
45,-25
(推定スピーカー角度,推定マイク角度)[43, -34]
45,25
(推定スピーカー角度,推定マイク角度)[43, 21]